# Citation Revenue

## Executive Summary

The daily citation revenue on and after October 15th, 2020 is significantly greater than the average daily revenue.

---

## Acquire
1. Aquire the dataset, [Los Angeles Parking Citations](https://www.kaggle.com/cityofLA/los-angeles-parking-citations).
1. Import libraries.
1. Load the dataset using pandas.
1. Display the shape and first/last 2 rows.
1. Display general statistics of the dataset - w/ the # of unique values in each column.
1. Uncover the number of missing values in each column.

## Prepare
- Remove spaces + capitalization from each column name.
- Drop features missing >=74.42\% of their values. 
- Drop unused features: `vin`, `rp_state_plate`, `make`, `body_style`, `color`, `marked_time`, `color_description`, `body_style_description`, `agency_description`, `meter_id`, `ticket_number`, and `violation_code`.
- Cast `issue_date` and `issue_time` to datetime data types.
- Transform `latitude` and `longitude` coordinates from NAD1983StatePlaneCaliforniaVFIPS0405 feet projection to EPSG4326 World Geodetic System 1984.
- Filter dates from 2017-01-01 to 2021-04-12.
- Filter citation issue times from 7:30am to 3:30pm.
- Filter for street sweeping citations.

In [2]:
# Import libraries
import pandas as pd
from scipy import stats
import plotly.express as px

import src

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load the data
data = src.get_sweep_data()
df = src.prep_sweep_data(data=data)

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

In [ ]:
df.info()

In [ ]:
# Create subplots of routes with the same sweep times.
px.histogram(df.issue_time.sort_values())

# Exploration: REDO ALL VISUALS
- Use plotly
- Move code to `explore.py`

## How much daily revenue is generated from street sweeper citations?
### Daily Revenue from Street Sweeper Citations
The number of street sweeping citations increased in October 2020.

In [ ]:
# Daily street sweeping citation revenue
daily_revenue = df.groupby('issue_date').fine_amount.sum()
daily_revenue.index = pd.to_datetime(daily_revenue.index)

In [ ]:
sns.set_context('talk')

# Plot daily revenue from street sweeping citations
daily_revenue.plot(figsize=(14, 7), label='Revenue', color='DodgerBlue')
plt.axhline(daily_revenue.mean(), color='black', label='Average Revenue')

plt.title("Daily Revenue from Street Sweeping Citations")
plt.xlabel('')
plt.ylabel("Revenue (in thousand's)")

plt.xticks(rotation=0, horizontalalignment='center', fontsize=13)
plt.yticks(range(0, 1_000_000, 200_000), ['$0', '$200', '$400', '$600', '$800',])
plt.ylim(0, 1_000_000)

plt.legend(loc=2, framealpha=.8);

### Anomaly: Declaration of Local Emergency

In [ ]:
sns.set_context('talk')

# Plot daily revenue from street sweeping citations
daily_revenue.plot(figsize=(14, 7), label='Revenue', color='DodgerBlue')
plt.axvspan('2020-03-16', '2020-10-14', color='grey', alpha=.25)
plt.text('2020-03-29', 890_000, 'Declaration of\nLocal Emergency', fontsize=11)


plt.title("Daily Revenue from Street Sweeping Citations")
plt.xlabel('')
plt.ylabel("Revenue (in thousand's)")

plt.xticks(rotation=0, horizontalalignment='center', fontsize=13)
plt.yticks(range(0, 1_000_000, 200_000), ['$0', '$200', '$400', '$600', '$800',])
plt.ylim(0, 1_000_000)

plt.legend(loc=2, framealpha=.8);

In [ ]:
sns.set_context('talk')

# Plot daily revenue from street sweeping citations
daily_revenue.plot(figsize=(14, 7), label='Revenue', color='DodgerBlue')
plt.axhline(daily_revenue.mean(), color='black', label='Average Revenue')
plt.axvline(datetime.datetime(2020, 10, 15), color='red', linestyle="--", label='October 15, 2020')

plt.title("Daily Revenue from Street Sweeping Citations")
plt.xlabel('')
plt.ylabel("Revenue (in thousand's)")

plt.xticks(rotation=0, horizontalalignment='center', fontsize=13)
plt.yticks(range(0, 1_000_000, 200_000), ['$0', '$200K', '$400K', '$600K', '$800K',])
plt.ylim(0, 1_000_000)

plt.legend(loc=2, framealpha=.8);

In [ ]:
## Add section comparing citation time distributions before and after parking was enforced.

Integrate articles:
- https://www.lamag.com/citythinkblog/street-sweeping-cutbacks/
Did the city go back to street sweeping every week?

Using the system, at www.streetsla.lacity.org/sweeping, residents can input an address and be notified by email 24 hours before the sweeper will be coming through to avoid a ticket. Residents can also call 311 for information or use the My311 app.
- https://www.streetsla.lacity.org/street-maintenance-division


# Results